In [35]:
#!pip install pandas datatable rpy2 matplotlib glmnet

In [74]:
import rpy2.robjects as robjects
import numpy as np
from rpy2.robjects import pandas2ri
import pandas as pd
pandas2ri.activate()
from glmnet import LogitNet
import scipy
from sklearn.linear_model import ElasticNet, SGDClassifier

In [37]:
faces = pd.read_csv('faces_smaller_chunk.csv', index_col=0)

In [38]:
faces.head()

,userid,gender,age,country,facial_hair,pol,pol_dat_us,pol_dat_ca,pol_dat_uk,pol_fb_us,...,left_eye_status.no_glass_eye_open,left_eye_status.normal_glass_eye_close,left_eye_status.dark_glasses,right_eye_status.normal_glass_eye_open,right_eye_status.no_glass_eye_close,right_eye_status.occlusion,right_eye_status.no_glass_eye_open,right_eye_status.normal_glass_eye_close,right_eye_status.dark_glasses,ethnicity.value
1,1,0.0,NaN,united states,0.001713,liberal,NaN,NaN,NaN,0.0,...,1.8,0.0,0.0,0.0,0.0,99.9,0.1,0.0,0.0,black
2,2,1.0,NaN,united states,0.005151,liberal,NaN,NaN,NaN,0.0,...,100.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,black
3,3,0.0,NaN,united states,0.977098,conservative,NaN,NaN,NaN,1.0,...,0.3,0.0,15.5,39.7,0.0,0.0,0.0,0.0,60.2,india
4,4,1.0,24.36,united states,0.000478,liberal,NaN,NaN,NaN,0.0,...,98.9,0.0,0.5,0.0,0.0,10.2,81.7,0.0,8.1,black
5,5,1.0,NaN,united states,0.005617,liberal,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,2.3,0.0,97.3,0.1,0.1,0.2,asian


In [39]:
ground_truth = ["pol_dat_us", "pol_dat_ca","pol_dat_uk","pol_fb_us"]

In [40]:
cv_folds = 4

In [41]:
folds = np.random.randint(1,cv_folds+1, len(faces))

VGG is a concatenation of facial features generated from Face++

In [42]:
vgg = pd.read_csv('./vgg_chunks/chunk_1.csv', index_col=0, header=0);

In [43]:
vgg.shape

(10859, 2048)

In [44]:
col = f'pred_{ground_truth[0]}'
col

'pred_pol_dat_us'

In [45]:
faces[col] = np.nan

In [46]:
faces[col]

1       NaN
2       NaN
3       NaN
4       NaN
5       NaN
         ..
10855   NaN
10856   NaN
10857   NaN
10858   NaN
10859   NaN
Name: pred_pol_dat_us, Length: 10859, dtype: float64

In [47]:
i = ground_truth[3]
j = 1

In [48]:
not_na = faces[faces[i].notna()].index
not_na

Int64Index([    1,     2,     3,     4,     5,     6,     7,     8,     9,
               10,
            ...
            10844, 10845, 10846, 10847, 10848, 10849, 10850, 10851, 10852,
            10853],
           dtype='int64', length=9429)

In [49]:
test_index = not_na[folds[not_na] == j]

train_index = not_na[folds[not_na] != j]

In [50]:
x=vgg.iloc[train_index]
x.shape

(7057, 2048)

In [51]:
y=faces[i][train_index]
y.head()

2    0.0
4    0.0
5    0.0
6    0.0
8    1.0
Name: pol_fb_us, dtype: float64

In [52]:
x.shape, y.shape

((7057, 2048), (7057,))

In [57]:
(x.values).shape

(7057, 2048)

In [77]:
m = SGDClassifier(penalty='elasticnet')

In [78]:
m.fit(x, y)

SGDClassifier(penalty='elasticnet')

In [79]:
vgg.iloc[test_index].values.shape

(2372, 2048)

In [84]:
faces[col][test_index] = m.predict(vgg.iloc[test_index])

<ipython-input-84-71efe33e940c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  faces[col][test_index] = m.predict(vgg.iloc[test_index])


In [85]:
faces[faces[col].notna()]

,userid,gender,age,country,facial_hair,pol,pol_dat_us,pol_dat_ca,pol_dat_uk,pol_fb_us,...,left_eye_status.normal_glass_eye_close,left_eye_status.dark_glasses,right_eye_status.normal_glass_eye_open,right_eye_status.no_glass_eye_close,right_eye_status.occlusion,right_eye_status.no_glass_eye_open,right_eye_status.normal_glass_eye_close,right_eye_status.dark_glasses,ethnicity.value,pred_pol_dat_us
1,1,0.0,NaN,united states,0.001713,liberal,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,99.9,0.1,0.0,0.0,black,0.0
3,3,0.0,NaN,united states,0.977098,conservative,NaN,NaN,NaN,1.0,...,0.0,15.5,39.7,0.0,0.0,0.0,0.0,60.2,india,0.0
7,7,1.0,27.79,united states,0.055370,liberal,NaN,NaN,NaN,0.0,...,0.0,6.2,0.0,0.0,0.0,100.0,0.0,0.0,white,0.0
9,9,1.0,38.13,united states,0.003721,conservative,NaN,NaN,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,white,1.0
19,19,1.0,NaN,united states,0.025718,liberal,NaN,NaN,NaN,0.0,...,0.0,0.0,0.1,0.0,0.0,99.9,0.0,0.0,black,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10835,10835,1.0,25.50,united states,0.000022,liberal,NaN,NaN,NaN,0.0,...,0.0,0.2,0.0,0.0,0.5,99.3,0.0,0.2,white,0.0
10836,10836,1.0,25.14,united states,0.000036,liberal,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,white,0.0
10845,10845,0.0,32.80,united states,0.998876,liberal,NaN,NaN,NaN,0.0,...,0.0,100.0,0.0,0.0,0.0,0.0,0.0,100.0,india,0.0
10851,10851,0.0,39.70,united states,0.328045,conservative,NaN,NaN,NaN,1.0,...,0.0,99.8,0.0,0.0,0.0,0.0,0.0,100.0,white,1.0
